In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime

from plotnine import *
theme_set(theme_matplotlib())

In [ ]:
hommik = 21_600 / 3600  # hommikune äratus kell 06:00
reservaeg = 60_300 / 3600  # reservaja algus kell 16:45
esimene_päev = 197  # teenistuse esimene päev 17.07.2023

In [ ]:
df = pd.read_csv('andmed/spioonid.csv')

df['kuupäev'] = df['kuupäev'].apply(lambda x: datetime.fromisoformat(x))
df['tund'] = df['kuupäev'].apply(lambda x: (x - x.replace(hour=0, minute=0, second=0)).total_seconds() / 3600)
df['päev'] = df['kuupäev'].apply(lambda x: x.timetuple().tm_yday - esimene_päev)
df['päev_n'] = df.groupby('päev').cumcount()
df['tühemik'] = df.groupby('päev')['tund'].diff().fillna(df['tund'] - hommik)
df['tegevusaeg'] = (df['tund'] >= reservaeg).map({True: 'Vaba- ja reservaeg', False:'Muu aeg'})

df.tail()

In [ ]:
g = ggplot(df) + facet_wrap('tegevusaeg', scales='free_x') + aes('tund', 'päev')
g += theme_linedraw() + theme(panel_spacing=0.05)
g += labs(x='Tund $(t)$', y='SBK päev')
g += geom_point(shape='x')

g.save('joonised/punktiprotsessid.png', dpi=100)
g